In [1]:
#versione del 11/12/2018 con (modifiche in ordine cronologico):
#- AnswerDuration rettificata con lo storno delle API Call dal BotAnswerDuration
#- nuovo campo "EventHourMinute"
#- 29/11/2018: nuovo codice per i Dataframe df_bot_messages e df_user_message
#- 01/12/2018: riorganizzazione delle scritture (prima DW e poi delta Table) per evitare i duplicati e update a type = "previous" dei record dopo la scrittura in Delta Table
#- 01/12/2018: delete dalla Delta Table "Events" per i record dello slot orario che si sta processando, così non ci sono problemi di duplicati in caso di riprocessamento
#- 03/12/2018: mapping del campo "envionment" nel flusso fino alla Delta Table "events" e al Datawarehouse
#- 05/12/2018: TXTAnalytics cast from string to float and DF filtered out from EventName != ServiceProfilerSample
#- 05/12/2018: added new field "EventNameDecode"
#- 06/12/2018: added df_5_final.persist()
#- 11/12/2018: commentati i persist/unpersist, valorizzato a false spark.databricks.queryWatchdog.enabled , spostata la scrittura sulla delta table prima della scrittura su DWH, inserito filtro sulle lettura della Delta EventsMirror (source) per non processare la data ed ora del RUN

#VERSIONE DEL 19/12/2018: corretto l'errore per cui ConversationStartDate, ConversationStartTime e ConversationEndTime avevano Null. Ora vengono valorizzati correttamente

In [2]:
%run "./Include/ETL_Main_Setup"

In [3]:
dbutils.widgets.text("input", "","")
dbutils.widgets.get("input")
datetimefolders = getArgument("input")
print(datetimefolders)
# prova 2018-11-12/11  per il Windowing, 2018-11-12/14 per bot answerduration. Oppure 2018-12-13/Federico con Pippo e ALDO. o i 3 files vecchi in Federico_old

2018-12-13/Federico

In [4]:
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "c9463783-977a-4fd6-be5e-685771eee8da")
spark.conf.set("dfs.adls.oauth2.credential", "bUB7Yo1o7M0e+yU0eNkT44cmRARwtjXSAGc7mR2qP9c= ")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/6815f468-021c-48f2-a6b2-d65c8e979dfb/oauth2/token")

databasetablePath = "adl://timvadevdls.azuredatalakestore.net/databricks/"

In [5]:
spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")

mount_storage(storage_account_name,storage_account_access_key)

In [7]:
%sql 
SELECT count(*)
FROM DEV.Events_Federico         -- Ho fatto una copia della Delta Table

count(1)
1021


In [8]:
#### Table Conversation was wrongly overwritten and in this run has got only 5 rows. But the pipeline is correct

%sql
SELECT count(*)
FROM DEV.Conversations

count(1)
5


In [9]:
dtf = "2018-10-02/"
hh = list(range(0,23))

#path = file_location + datetimefolders +  "/*.blob"
parser_wo = AppInsightEventParser(storage_account_name,container_name_wo,folder_name_wo)
parser_bu = AppInsightEventParser(storage_account_name,container_name_bu,folder_name_bu)

parser_wo.addStage(Stage1())
parser_bu.addStage(Stage1())
#parser_wo.addStage(Stage2("/delta/DEV/events/"))
#parser_wo.addStage(Stage3())
#parser_wo.addStage(Stage4())

#dw_info_wo = DWInfo(dwh_server,dwh_db_name,"timsqladmin@tim-va-dev-sqlserver","P@ssword!")
#parser_wo.addStage(Stage5(dw_info_wo))


#datetimefolders = dtf + str(hh)
df_wo = parser_wo.parseBlob(datetimefolders,parseRow)
#df_wo.printSchema()
#display(df_wo)
df_bu = parser_bu.parseBlob(datetimefolders,parseRow)
#df_bu.printSchema()
#display(df_bu)
#df_co = parser_co.parseBlob(datetimefolders,parseRow)
#display(df_bu)
# 2018-11-19 Simone Begin ->
df_wo = df_wo.withColumn("applicationInsight", F.lit(container_name_wo))
df_wo = df_wo.withColumn("businessUnit", F.lit(businessUnitName_wo))
df_bu = df_bu.withColumn("applicationInsight", F.lit(container_name_bu))
df_bu = df_bu.withColumn("businessUnit", F.lit(businessUnitName_bu))
#df_co = df_co.withColumn("applicationInsight", F.lit(container_name_co))
#df_co = df_co.withColumn("businessUnit", F.lit(businessUnitName_co))

""" AVOIDING PROBLEMS OF SCHEMA """
if df_wo.count()==0 and df_bu.count()> 0 :
    print('df_wo is empty')
    df_total= df_bu
elif df_bu.count()==0 and df_wo.count()> 0:
    print('df_bu is empty')
    df_total= df_wo
elif df_bu.count()>0 and df_wo.count()> 0:
    print("both wo and bu have records")
    df_total= df_wo.union(df_bu)
else:
    print("non c'è nessuno dei due path")

"""df_total = df_wo.union(df_bu)  """  # 3/12/2018 Federico: it was --> df_bu.union(df_wo)

# Union of previous united WO and BU dataframes with CO dataframe

#df_total = df_total.union(df_co)

"""display(df_total)"""
# 2018-11-19 Simone -> End
#df_total.printSchema()
df_1 = df_total
cols_pilot = df_1.columns
df_1 = df_1.withColumn("type", F.lit("new"))

#df.createOrReplaceTempView("temp")
#df.write.mode("append").format("delta").partitionBy("eventName").save("/mnt/delta/eventsdl")

#spark.sql("OPTIMIZE eventsdl ZORDER BY (conversationId)")

#display(df_1)

root
-- ClientAtHome: string (nullable = true)
-- LUISentities: string (nullable = true)
-- LUISintent: string (nullable = true)
-- LUISscore: string (nullable = true)
-- QNAanswer: string (nullable = true)
-- QNAcategory: string (nullable = true)
-- QNAscore: string (nullable = true)
-- QNAsource: string (nullable = true)
-- QNAuserMessage: string (nullable = true)
-- TXTANALYTICSscore: string (nullable = true)
-- activityId: string (nullable = true)
-- channel: string (nullable = true)
-- conversationId: string (nullable = true)
-- currentScenario: string (nullable = true)
-- diagnosticResponse: string (nullable = true)
-- dialogCloseType: string (nullable = true)
-- endTimeAPICall: string (nullable = true)
-- eventName: string (nullable = true)
-- eventTime: string (nullable = true)
-- eventdate: string (nullable = true)
-- isAuthenticated: string (nullable = true)
-- isLeaf: string (nullable = true)
-- itemId: string (nullable = true)
-- mainScenario: string (nullable = true)
-- modemAlignment: string (nullable = true)
-- modemChanged: string (nullable = true)
-- modemType: string (nullable = true)
-- notUnderstoodMessage: string (nullable = true)
-- problemSolved: string (nullable = true)
-- reason: string (nullable = true)
-- scenario: string (nullable = true)
-- scenarioCalled: string (nullable = true)
-- scenarioId: string (nullable = true)
-- startTimeAPICall: string (nullable = true)
-- telemetryId: string (nullable = true)
-- text: string (nullable = true)
-- ticket: string (nullable = true)
-- ticketForm: string (nullable = true)
-- timestamp: string (nullable = true)
-- toolExec: string (nullable = true)
-- toolResult: string (nullable = true)
-- user: string (nullable = true)
-- userId: string (nullable = true)
-- userName: string (nullable = true)
-- applicationInsight: string (nullable = true)
-- businessUnit: string (nullable = true)

df_bu is empty

In [10]:
#%fs rm -r mnt/delta/eventsdl

In [11]:
df_1= df_1.withColumn('TXTANALYTICSscore' , df_1['TXTANALYTICSscore'].cast(FloatType()))
df_1= df_1.select(['*']).where(col('EventName')!= 'ServiceProfilerSample')

In [12]:
df_1= df_1.withColumn('eventName', when(df_1.conversationId== 'ALDO', F.lit('MBFEvent.QNAEvent')).otherwise(df_1.eventName)).withColumn('QNAcategory', 
                                             when(df_1.conversationId == 'ALDO', F.lit('ChitChat')).otherwise(df_1.QNAcategory)) #modificata ad hoc per far sì che venga in corso
display(df_1) #4 record nello slot orario corrente

ClientAtHome,LUISentities,LUISintent,LUISscore,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,itemId,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,problemSolved,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type
,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,null,true,,06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new
,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,null,,,06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new
,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,null,true,,12345,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new
,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,null,,,6789,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new


In [13]:
df_1.count()  #921 perchè Events è vuota

Out[ 54 ]: 4

In [14]:
#tableName = "adl://timvadevdls.azuredatalakestore.net/delta/eventsdl/"

#spark.sql("OPTIMIZE '/mnt/delta/eventsdl' ZORDER BY (conversationId)")

In [15]:
#spark.read.format("delta").load(tableName).count()
#ccdf = spark.sql("SELECT * FROM delta.`/mnt/delta/eventsdl` WHERE eventName = 'MBFEvent.CallScenario' AND conversationId = 'BTvzSeL1ZzC7b1MZDplwoK'")

#display(ccdf)

In [16]:
def parseLuisEntity(x):
    
    rowDict = x.asDict()
    jsonList = rowDict.pop('LUISentities')

    js_d = json.loads(jsonList)
    for entity in js_d:
        rowDict['LuisEntityKey'] = entity["key"]
        rowDict['LuisEntityValue'] = entity["value"]
        rowDict['LuisEntityScore'] = entity["score"]
        yield Row(**rowDict)

                          
def timestamp_to_long(dt):
    """
    This function takes a timestamp and returns it as microseconds from epoc, respecting timezone.  If no tzinfo is specified,
    UTC is assumed.
    :param dt:
    :return:
    """
    if dt is None:
        return None

    if not dt.tzinfo:
        dt = dt.replace(tzinfo=datetime.timezone.utc)

    return int(
        ((dt - datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)) / datetime.timedelta(microseconds=1)))


def timestamp_diff(startTime, endTime):
  
    if startTime is None or endTime is None:
        return 0

    tt = timestamp_to_long(endTime) - timestamp_to_long(startTime)
    return tt / 1000

In [17]:
#ENTITIES MANAGEMENT
dfEntities = df_1.where((df_1["LUISentities"] != '') & (df_1["LUISentities"] != '[]')).select("conversationId","timestamp","itemId","LUISentities")
#flatMapping
mappedj = dfEntities.rdd.flatMap(parseLuisEntity)

#If entities have been found, then write them in the DWH
if(mappedj.isEmpty() == False):
    # Create a DataFrame from the mapped collection
    df_forEntitiesTable = spark.createDataFrame(mappedj)
    df_forEntitiesTable = df_forEntitiesTable.withColumn("DateUTCInsertRow", from_utc_timestamp(F.lit(current_timestamp()),"Europe/Rome")) #Andrea 25/10/2018
    df_forEntitiesTable = df_forEntitiesTable.select("conversationId","itemId","timestamp","LuisEntityKey","LuisEntityValue","LuisEntityScore","DateUTCInsertRow")
    # Entities written in dedicated table into the Datawarehouse: [EventEntity] table
    df_forEntitiesTable.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "EventEntity").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()
    else:
        print(mappedj.isEmpty())

True

In [18]:
print(datetimefolders)
datefolder = datetimefolders[0:10]
print(datefolder)
datehour = datetimefolders[11:13]
print(datehour)

2018-12-13/Federico
2018-12-13
Fe

In [19]:
#BEFORE READ, DELETE FROM DELTA TABLE
#spark.sql(" DELETE FROM delta.`/delta/{0}/Events/` WHERE EventDate||'/'||hour(EventTime) = '{1}' ".format(environment, datetimefolders))  
  
# Reading data from Delta Table (DBFS) to take data from the past
previous_events = spark.read.format("delta").load("/delta/"+environment+"/Events/").select(cols_pilot)

#il giorno 2018-12-13 ho commentato parte sotto perchè per qualche motivo non importava alcuna riga dalla delta, ovvero previous events è vuota
"""
#MODIFIED ON 2018-12-11 adding logic for filtering duplicates, not based on deleting infos, but filtering them and at the overwriting them at the end of the present notebook.
previous_events_filtered = spark.sql("SELECT * FROM {0}.Events WHERE eventDate != '{1}' AND hour(eventTime) != '{2}' ".format(environment, datefolder, datehour))
previous_events = previous_events_filtered.select(cols_pilot)
#previous_events_filtered.printSchema()
#previous_events = spark.sql("SELECT eventDate, hour(eventTime) as Hour FROM previous_events_table GROUP BY eventDate, hour(eventTime) ORDER BY eventDate, hour(eventTime)")"""
                                     #WHERE eventDate = '{1}' AND hour(EventTime) = '{2}' ".format(environment, datefolder, datehour))"""
#display(previous_events)
previous_events = previous_events.withColumn("type", F.lit("previous"))
# Union of two DataFrames
unionDataFrame = previous_events.unionAll(df_1)

In [20]:
previous_events.count()  #ho 8 record nella delta

Out[ 59 ]: 0

In [21]:
unionDataFrame.count()  #ha 8+4=12 records

Out[ 60 ]: 4

In [22]:
#insert apiTimeDiff calculated column
func_udf = udf(timestamp_diff, FloatType())
df_allEvents = unionDataFrame.withColumn("apiTimeDiff", when(unionDataFrame.eventName == 'MBFEvent.APICall', func_udf(unionDataFrame["startTimeAPICall"], unionDataFrame["endTimeAPICall"])))
df_allEvents.createOrReplaceTempView('allEvents')
display(df_allEvents)

ClientAtHome,LUISentities,LUISintent,LUISscore,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,itemId,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,problemSolved,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff
,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,null,true,,06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null
,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,null,,,06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null
,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,null,true,,12345,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null
,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,null,,,6789,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null


In [23]:
df_allEvents.count()

Out[ 62 ]: 4

In [ ]:
# Expaining Windowing function
"""Per ogni ActivityId di ogni conversationId voglio calcolare la botAnswerDuration, intesa come l'intervallo di tempo che intercorre tra il primo messaggio dell'utente e la prima risposta del Bot. Per cui Per ogni activityId prendo questi due eventi, MBFEvent.UserMessage e MBFEvent.BotMessage, dove il BotMessage deve essere in un eventTime successivo, e avrò una situazione del tipo:
ActivityId    EventName    EventTime
   A           UserMessage    15:30
   A           BotMessage     15:35
 Adesso uso il Windowing per 'spostare' l eventTime del record dell UserMessage sulla riga successiva, in modo che poi posso fare la sottrazione. ovvero avrò:
 ActivityId    EventName    EventTime  EventTimePrevious   BotAnswerDuration
   A           UserMessage    15:30         /                    /
   A           BotMessage     15:35         15:35               00:05
   
   L'ho 'trascinato' sotto, quindi Windowing crea una finestra di un record, nel senso che non guardo solo il un record singolo ma anche quello prima. Avendoli ora sulla stessa riga, con un semplice withColumn posso calcolare BotAnswerDuration facendo la sottrazione.
   Infine vado in join left con Events, perchè questo Windowing è fatto su un sottoinsieme di tutti i record, ovvero solo quelli corrispondenti agli eventi UserMessage e BotMessage (Es: Se Events ha 1000 record, Windowing lavora su 150. Facendo Events.join(df_Windowing, how='left') riottengo tutti i 1000 record con il nuovo campo botAnswerDuration che sarà valorizzato solo su alcuni dei 150 (sugli eventi BotMessage dei 150)). """

In [25]:
#DATAFRAME with ONLY userMessages and botMessage for calculating responseTime
df_bot_messages = spark.sql(" \
          SELECT A.ConversationId, A.ActivityId, A.eventName, A.itemId,  A.eventTime \
          FROM allEvents A \
          INNER JOIN ( \
                            SELECT AA.itemId, AA.ActivityId \
                            FROM allEvents AA \
                            INNER JOIN ( \
                                        SELECT ActivityId ,min(eventTime) as eventTime \
                                        FROM allEvents \
                                        WHERE eventName = 'MBFEvent.BotMessage' and ActivityId is not null \
                                        GROUP BY ActivityId \
                                    ) BB \
                              on AA.ActivityId = BB.ActivityId and AA.eventTime = BB.eventTime \
                            GROUP BY AA.itemId, AA.ActivityId \
                    ) B \
           ON A.ActivityId = B.ActivityId and A.itemId = B.itemId \
           WHERE A.eventName = 'MBFEvent.BotMessage' \
          ")

df_user_messages = spark.sql(" \
          SELECT A.ConversationId, A.ActivityId, A.eventName, A.itemId, B.eventTime \
          FROM allEvents as A \
          INNER JOIN ( \
                      SELECT itemId, ActivityId ,min(eventTime) as eventTime \
                      FROM allEvents \
                       WHERE eventName = 'MBFEvent.UserMessage' and ActivityId is not null \
                      GROUP BY itemId, ActivityId \
                  ) B \
          ON A.itemId = B.itemId \
	       ")

df_all_messages = df_bot_messages.union(df_user_messages).orderBy('ConversationId', 'activityId', 'eventTime', 'itemId')

# TimeDiff between UserMessage and BotMessage and insert botTimeDuration in df_2_time new dataframe
df_all_messages = df_all_messages.select("ConversationId","ActivityId","itemId","eventName","eventTime").orderBy("conversationid","eventTime")
display(df_all_messages)

ConversationId,ActivityId,itemId,eventName,eventTime
6LRLPjJDLkQ85QplizAeFS,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,06470259-e671-11e8-88dc-9fb96e7465b9,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000


In [26]:
time_window = Window.partitionBy("activityId").orderBy("ActivityId", "eventTime")
df_all_messages = df_all_messages.withColumn("prev_time", F.lag(df_all_messages.eventTime).over(time_window))
df_all_messages = df_all_messages.withColumn("eventTime", df_all_messages["eventTime"].cast(TimestampType()))
df_all_messages = df_all_messages.withColumn("prev_time", df_all_messages["prev_time"].cast(TimestampType()))
df_all_messages = df_all_messages.withColumn("BotAnswerDuration", func_udf(df_all_messages["prev_time"], df_all_messages["eventTime"]))
# df_all_messages = df_all_messages.withColumn("BotAnswerDuration", func_udf(df_all_messages["BotAnswerDuration"], df_all_messages["apiTimeDiff"]))
df_all_messages = df_all_messages.withColumn("EventTimePrevious", to_timestamp(df_all_messages["prev_time"]) ) #add column with Date of Previous Message, to do mapping of field also in DW
#display(df_all_messages).groupby('ConversationId', 'ActivityId').select(['*'])
df_with_BotTimeDuration = df_all_messages.select('itemId', 'BotAnswerDuration', 'EventTimePrevious').where(col("BotAnswerDuration").isNotNull())

#insert BotTimeDuration to a new dataframe
df_2_time = df_allEvents.join(df_with_BotTimeDuration, 'itemId', 'left')
display(df_2_time)
#display(df_2_time.filter(df_2_time.activityId == 'JEBXXIfay3m328mWQLvL0x|0000160').select('eventName', 'eventTime', 'BotAnswerDuration', 'apiTimeDiff', 'EventTimePrevious'))

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,problemSolved,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious
6789,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,null,,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,null,true,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null
12345,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,null,true,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,null,,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null


In [27]:
#PREPARE DATA TO BE WRITTEN ENRICHED DATA INTO DWH Tables and STORE it in Databricks TABLES
df_3_final = df_2_time
df_3_final = df_3_final.withColumn('eventdate', df_3_final['eventTime'].cast('string').substr(1,10)) #cast from date to string and take left 10 characters to have a string in format "yyyy-mm-dd"
df_3_final = df_3_final.withColumn("DateUTCInsertRow", from_utc_timestamp(F.lit(current_timestamp()),"Europe/Rome"))


#2018-11-05 Simone/Federico
from types import ClassType
ClassType(1)
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, NullType
udf_to_float = UserDefinedFunction(lambda x: float(0) if ClassType(x) != float else x, FloatType() )    
df_4_final = df_3_final.select(*[udf_to_float(column).alias('BotAnswerDuration') if column == 'BotAnswerDuration' else column for column in df_3_final.columns])
"""METTERE QUESTA SOTTO (df_5_final) SOLAMENTE SE VOGLIO CAMBIARE I NULL DI APITIMEDIFF CON DEGLI ZERO"""
#df_5_final = df_4_final.select(*[udf_to_float(column).alias('apiTimeDiff') if column == 'apiTimeDiff' else column for column in df_4_final.columns])#
#2018-11-05 Simone/Federico    

Out[ 66 ]: 'METTERE QUESTA SOTTO (df_5_final) SOLAMENTE SE VOGLIO CAMBIARE I NULL DI APITIMEDIFF CON DEGLI ZERO'

In [28]:
df_2_time.count()   #vengono creati ERRONAMENTE PIù RECORD col windowing. Debuggare

Out[ 67 ]: 4

###### Selection of records in df_4_final having BotAnswerDuration > 0

In [30]:
df_4_final.count()

Out[ 68 ]: 4

In [31]:
df_4_final.createOrReplaceTempView('table_4_final')

#df_5_final.printSchema()

df_bot_responseTime = spark.sql(" \
          SELECT ConversationId as botConversationId, ActivityId as botActivityId, eventName as botEventName, BotAnswerDuration as botBotAnswerDuration, (apiTimeDiff) as botApiTimeDiff, eventTime as botEventTime, startTimeAPICall as botStartTimeAPICall, endTimeAPICall as botEndTimeAPICall  \
          FROM table_4_final \
          WHERE BotAnswerDuration > 0 AND eventName = 'MBFEvent.BotMessage' \
          GROUP BY ConversationId, ActivityId, eventName, BotAnswerDuration, apiTimeDiff, eventTime, startTimeAPICall, endTimeAPICall ORDER BY eventTime  \
          ")
display(df_bot_responseTime)

botConversationId,botActivityId,botEventName,botBotAnswerDuration,botApiTimeDiff,botEventTime,botStartTimeAPICall,botEndTimeAPICall


#### Selection of records in df_4_final having eventName = 'MBFEvent.APICall'

In [33]:
df_api_responseTime = spark.sql(" \
          SELECT ConversationId as apiConversationId, ActivityId as apiActivityId, eventName as apiEventName, BotAnswerDuration as apiBotAnswerDuration, (apiTimeDiff) as apiApiTimeDiff, eventTime as apiEventTime, startTimeAPICall as apiStartTimeAPICall, endTimeAPICall as apiEndTimeAPICall  \
          FROM table_4_final \
          WHERE eventName = 'MBFEvent.APICall' \
          GROUP BY ConversationId, ActivityId, eventName, BotAnswerDuration, apiTimeDiff, eventTime, startTimeAPICall, endTimeAPICall ORDER BY eventTime  \
          ")
display(df_api_responseTime)

apiConversationId,apiActivityId,apiEventName,apiBotAnswerDuration,apiApiTimeDiff,apiEventTime,apiStartTimeAPICall,apiEndTimeAPICall


##### Join tra Bot e api

In [35]:
df_join_responseTime = df_bot_responseTime.join(df_api_responseTime, ((df_bot_responseTime.botConversationId == df_api_responseTime.apiConversationId) & ((df_bot_responseTime.botActivityId == df_api_responseTime.apiActivityId))), 'left')
#df_bot_responseTime.unpersist() # unpersist() useful to prevent duplicates
#df_api_responseTime.unpersist()
df_join_responseTime_final = df_join_responseTime.where((col("botEventTime") >= col("apiEndTimeAPICall")) ).orderBy('apiEventTime') # Checking that the call of API  ends before the BotAnswer for the same ActivityId
#df_join_responseTime.unpersist()
df_join_responseTime_final= df_join_responseTime_final.select('botConversationId',
 'botActivityId',
 'botEventName',
 'botBotAnswerDuration',
 'apiApiTimeDiff',
 'botEventTime',
 'apiEventTime',
 'apiStartTimeAPICall',
 'apiEndTimeAPICall')
#display(df_join_responseTime_final.select('botConversationId','botActivityId','botEventTime','apiStartTimeAPICall','apiEndTimeAPICall'))

display(df_join_responseTime_final)

""" Replacing NULLS with zeros"""  # <-- LASCIARLO SOLO SE VOGLIO CAMBIARE I NULL DI APITIMEDIFF CON DEGLI ZERO!!!!
"""udf = UserDefinedFunction(lambda x: float(0) if ClassType(x) != float else x, FloatType() )
df_join_responseTime_final= df_join_responseTime_final.select(*[udf(column).alias('apiApiTimeDiff') if column == 'apiApiTimeDiff' else column for column in df_join_responseTime_final.columns])"""

botConversationId,botActivityId,botEventName,botBotAnswerDuration,apiApiTimeDiff,botEventTime,apiEventTime,apiStartTimeAPICall,apiEndTimeAPICall


##### Sum apiTimeDiff for the same activity: subtracting sum from BotAnswerDuration

In [37]:
df_join_responseTime_final.createOrReplaceTempView('table_join_final')

df_sum_api= spark.sql( "SELECT botConversationId, botActivityId, botEventName, botBotAnswerDuration,botEventTime, sum(apiApiTimeDiff) as sumApiTimeDiff\
            FROM table_join_final \
            GROUP BY botConversationId, botActivityId, botEventName, botBotAnswerDuration,botEventTime")


df_sum_api_final = df_sum_api.withColumn("botBotAnswerDuration", (df_sum_api["botBotAnswerDuration"] - df_sum_api["sumApiTimeDiff"]))
df_sum_api_final= df_sum_api_final.drop("sumApiTimeDiff")
display(df_sum_api)

botConversationId,botActivityId,botEventName,botBotAnswerDuration,botEventTime,sumApiTimeDiff


In [38]:
display(df_sum_api_final)

botConversationId,botActivityId,botEventName,botBotAnswerDuration,botEventTime


##### Updating df_5_final with the rectified BotAnswerDuration (same schema)

In [40]:
df_5_final = df_4_final.join(df_sum_api_final, (df_4_final.conversationId == df_sum_api_final.botConversationId) & (df_4_final.activityId == df_sum_api_final.botActivityId) & (df_4_final.eventName == df_sum_api_final.botEventName) & (df_4_final.eventTime == df_sum_api_final.botEventTime), 'left').select('itemId',
 'ClientAtHome',
 'LUISentities',
 'LUISintent',
 'LUISscore',
 'ProblemSolved',
 'QNAanswer',
 'QNAcategory',
 'QNAscore',
 'QNAsource',
 'QNAuserMessage',
 'TXTANALYTICSscore',
 'activityId',
 'channel',
 'conversationId',
 'currentScenario',
 'diagnosticResponse',
 'dialogCloseType',
 'endTimeAPICall',
 'eventName',
 'eventTime',
 'eventdate',
 'isAuthenticated',
 'isLeaf',
 'mainScenario',
 'modemAlignment',
 'modemChanged',
 'modemType',
 'notUnderstoodMessage',
 'reason',
 'scenario',
 'scenarioCalled',
 'scenarioId',
 'startTimeAPICall',
 'telemetryId',
 'text',
 'ticket',
 'ticketForm',
 'timestamp',
 'toolExec',
 'toolResult',
 'user',
 'userId',
 'userName',
 'applicationInsight',
 'businessUnit',
 'type',
 'apiTimeDiff',
 'BotAnswerDuration',
 'botBotAnswerDuration',
 'EventTimePrevious',
 'DateUTCInsertRow').orderBy('eventTime')

#df_4_final.unpersist()   #avoiding duplicates
# relevant columns to display: .select('conversationId', 'activityId', 'botBotAnswerDuration', 'BotAnswerDuration', 'apiTimeDiff', 'eventTime' ).orderBy('eventTime'))

#there is the original BotAnswerDuration to make comparisons

display(df_5_final)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,botBotAnswerDuration,EventTimePrevious,DateUTCInsertRow
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,0.0,null,null,2018-12-19T20:25:12.420+0000
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,0.0,null,null,2018-12-19T20:25:12.420+0000
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,0.0,null,null,2018-12-19T20:25:12.420+0000
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,0.0,null,null,2018-12-19T20:25:12.420+0000


In [41]:
#display(df_5_final.select(['botBotAnswerDuration', 'BotAnswerDuration' ]))   #comparison between original and new BotAnswerDuration
df_5_final= df_5_final.withColumn('botBotAnswerDuration', df_5_final['botBotAnswerDuration'].cast(FloatType()))
df_5_final= df_5_final.withColumn('BotAnswerDuration', df_5_final['BotAnswerDuration'].cast(FloatType()))
#df_5_final.printSchema()

def bot_duration(old,new):
    if new is None and old>0:
        return old
    else:
        return new
  
udf_bot_duration= UserDefinedFunction(bot_duration, FloatType())
df_5_final= df_5_final.withColumn('BotAnswerDuration', udf_bot_duration(df_5_final['BotAnswerDuration'], df_5_final['botBotAnswerDuration']))
display(df_5_final)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,botBotAnswerDuration,EventTimePrevious,DateUTCInsertRow
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,null,2018-12-19T20:26:18.440+0000
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,null,2018-12-19T20:26:18.440+0000
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,null,2018-12-19T20:26:18.440+0000
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,null,2018-12-19T20:26:18.440+0000


##### With this cell it is over the rectification of BotAnswerDuration

In [43]:
df_5_final= df_5_final.drop('botBotAnswerDuration')
df_5_final.schema== df_4_final.schema   #check if the schema is the same of the original (the one before rectification)

Out[ 76 ]: False

In [44]:
#df_5_final.persist()

In [45]:
#06-12-2018 to be added Environment and eventNameDecode correctly
"""df_5_final = df_5_final.withColumn('Environment', F.lit(environment))
df_5_final = df_5_final.withColumn('eventNameDecode', when(df_5_final['eventName'].like('%MBFEvent%') , df_5_final['eventName'].substr(10,15)).otherwise(df_5_final['eventName']))"""

Out[ 78 ]: "df_5_final = df_5_final.withColumn('Environment', F.lit(environment))\ndf_5_final = df_5_final.withColumn('eventNameDecode', when(df_5_final['eventName'].like('%MBFEvent%') , df_5_final['eventName'].substr(10,15)).otherwise(df_5_final['eventName']))"

added Environment and eventNameDecode

In [47]:
df_5_final = df_5_final.withColumn('Environment', F.lit(environment))
df_5_final = df_5_final.withColumn('eventNameDecode', when(df_5_final['eventName'].like('%MBFEvent%') , df_5_final['eventName'].substr(10,15)).otherwise(df_5_final['eventName']))

In [48]:
df_5_final.count()

Out[ 80 ]: 4

### Adding new fields to Events: Status, Recognition, Status Code, Recognition Code following Business rules

In [50]:
#new_Events = df_5_final.filter(df_5_final.type == 'new')
new_Events= df_5_final
new_Events= new_Events.withColumn('Status', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('in corso')).when(new_Events.mainScenario == 'Default Dialog', F.lit('in corso')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('in corso')).otherwise(F.lit('undefined')))

new_Events= new_Events.withColumn('Recognition', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('riconosciuta')).when(new_Events.mainScenario == 'Default Dialog', F.lit('non riconosciuta')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('inconsistente')).otherwise(F.lit('undefined')))

new_Events= new_Events.withColumn('Status code', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('IC')).when(new_Events.mainScenario == 'Default Dialog', F.lit('IC')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('IC')).otherwise(F.lit('undefined')))

new_Events= new_Events.withColumn('Recognition code', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('RIC')).when(new_Events.mainScenario == 'Default Dialog', F.lit('NRIC')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('INC')).otherwise(F.lit('undefined')))

display(new_Events)
#new_Events.count()
#print(new_Events.explain())

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,Status code,Recognition code
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:28:23.955+0000,DEV,Sentiment,undefined,undefined,undefined,undefined
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:28:23.955+0000,DEV,UserMessage,undefined,undefined,undefined,undefined
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:28:23.955+0000,DEV,Sentiment,undefined,undefined,undefined,undefined
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:28:23.955+0000,DEV,QNAEvent,in corso,inconsistente,IC,INC


#####Adding to Events table a temporary field which says the difference (in hours) between the actual time and the EventTime

In [52]:
def timestamp_diff_hours(endTime,startTime):
    if startTime is None or endTime is None:
        return 0
    tt = timestamp_to_long(endTime) - timestamp_to_long(startTime)
    return tt / (1000)
  
def arrotonda(x):
    return round(x,2)
  
func_udf2= udf(timestamp_diff_hours, FloatType())
new_Events= new_Events.withColumn('diff_hours', func_udf2(current_timestamp(), new_Events['eventTime'])/(1000*3600))

udf_round= udf(arrotonda, FloatType())
new_Events= new_Events.withColumn('diff_hours', udf_round(new_Events['diff_hours']))

display(new_Events)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,Status code,Recognition code,diff_hours
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:29:09.469+0000,DEV,Sentiment,undefined,undefined,undefined,undefined,895.67
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:29:09.469+0000,DEV,UserMessage,undefined,undefined,undefined,undefined,895.67
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:29:09.469+0000,DEV,Sentiment,undefined,undefined,undefined,undefined,895.63
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:29:09.469+0000,DEV,QNAEvent,in corso,inconsistente,IC,INC,895.62


#### Filtering Events in standby (between 20 and 30 minutes) and after standby (more than 30 minutes)

In [54]:
new_Events.createOrReplaceTempView('new_Events_sql')

stand_by = spark.sql(" \
          SELECT ConversationId as convId, max(eventTime) as last_time_conv \
          FROM new_Events_sql \
          WHERE diff_hours*60 >= 20 AND diff_hours*60 <30 \
          GROUP BY ConversationId \
          ORDER BY max(eventTime)\
          ")

#ATTENZIONE: L'UPPER BOUND DELLO STANBY DEVE ESSERE IL LOWER BOUND DI "OLTRE_STAN_BY, E UNO CHE LO INCLUDE E L ALTRO NO, PER COPRIRE TUTTI I CASI"
after_stand_by = spark.sql(" \
          SELECT ConversationId as convId, max(eventTime) as last_time_conv \
          FROM new_Events_sql \
          WHERE diff_hours*60 >= 30 \
          GROUP BY ConversationId \
          ORDER BY max(eventTime)\
          ")

stand_by= stand_by.withColumn('Status_temp', F.lit('stand by'))
after_stand_by= after_stand_by.withColumn('Status_temp', F.lit('oltre stand by'))
union_sb_and_after= stand_by.unionAll(after_stand_by)
display(union_sb_and_after)  #avrò l ultimo record di ogni conv che è oltre lo standby, ovvero oltre i 30 min

convId,last_time_conv,Status_temp
6LRLPjJDLkQ85QplizAeFS,2018-11-12T11:49:04.561+0000,oltre stand by
PIPPO,2018-11-12T11:51:04.544+0000,oltre stand by
ALDO,2018-11-12T11:52:08.561+0000,oltre stand by


####Joining table Events with the one filtered in the previous cell, referring to all conversations in SB and after

In [56]:
new_Events2= new_Events.join(union_sb_and_after, ((new_Events.conversationId == union_sb_and_after.convId) & (new_Events.eventTime == union_sb_and_after.last_time_conv)
                        ), 'left')

new_Events2= new_Events2.withColumn('Status', when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] == 'true')), F.lit('chiusa')).when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] !='true')), F.lit('abbandonata')).otherwise(new_Events2['Status'])).withColumn('Status code', when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] == 'true')), F.lit('CH')).when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] !='true')), F.lit('AB')).otherwise(new_Events2['Status code']))

new_Events2= new_Events2.withColumnRenamed("Status code", "StatusCode").withColumnRenamed("Recognition code", "RecognitionCode")

#display(new_Events2.select('isLeaf','Status_temp','Status','StatusCode', 'Recognition', col('RecognitionCode').alias('ProvaRename')))
display(new_Events2)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,diff_hours,convId,last_time_conv,Status_temp
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:31:38.622+0000,DEV,Sentiment,abbandonata,undefined,AB,undefined,895.68,PIPPO,2018-11-12T11:51:04.544+0000,oltre stand by
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:31:38.622+0000,DEV,Sentiment,undefined,undefined,undefined,undefined,895.71,null,null,null
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:31:38.622+0000,DEV,QNAEvent,abbandonata,inconsistente,AB,INC,895.66,ALDO,2018-11-12T11:52:08.561+0000,oltre stand by
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:31:38.622+0000,DEV,UserMessage,abbandonata,undefined,AB,undefined,895.71,6LRLPjJDLkQ85QplizAeFS,2018-11-12T11:49:04.561+0000,oltre stand by


In [57]:
new_Events2.count()

Out[ 85 ]: 4

In [58]:
#display(new_Events2.where(col('conversationId') == 'BEDTFSDcWqa6rfGc9PwiKP').orderBy('eventTime'))   
display(new_Events2.filter((new_Events2.Status=='abbandonata') | (new_Events2.Status=='chiusa'))) 
# equivalente: display(new_Events2.filter(new_Events2.Status_temp=='oltre stand by')), perchè chiusa o abbandonata sono oltre sb

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,diff_hours,convId,last_time_conv,Status_temp
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:35:12.866+0000,DEV,Sentiment,abbandonata,undefined,AB,undefined,895.74,PIPPO,2018-11-12T11:51:04.544+0000,oltre stand by
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:35:12.866+0000,DEV,QNAEvent,abbandonata,inconsistente,AB,INC,895.72,ALDO,2018-11-12T11:52:08.561+0000,oltre stand by
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:35:12.866+0000,DEV,UserMessage,abbandonata,undefined,AB,undefined,895.77,6LRLPjJDLkQ85QplizAeFS,2018-11-12T11:49:04.561+0000,oltre stand by


In [59]:
new_Events2= new_Events2.drop('convId', 'last_time_conv', 'Status_temp')
new_Events2= new_Events2.withColumnRenamed("diff_hours", "HourFromLastEvent")

In [60]:
new_Events2= new_Events2.withColumn('EventHourMinute' , new_Events2['eventTime'].cast(StringType()))
#display(new_Events2)

In [61]:
def hour_minute(x):
    return  ''.join(x.split()[1][:5].split(':'))                         

udf_hour_minute= UserDefinedFunction(hour_minute, StringType())
new_Events2= new_Events2.withColumn('EventHourMinute' , udf_hour_minute(new_Events2['EventHourMinute']))
display(new_Events2)
#new_Events2.persist() #riabilitato alle 19:38 del 30/11/2018 dopo il Run Manuale del 2018-11-30/10 partito alle 19:30 (osserviamo se cambia qualcosa con il successivo RUN)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,HourFromLastEvent,EventHourMinute
6789,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:51:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:37:06.832+0000,DEV,Sentiment,abbandonata,undefined,AB,undefined,895.77,1151
06470258-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,0.5085391,,pat,PIPPO,,,,null,MBFEvent.Sentiment,2018-11-12T11:49:04.544+0000,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:37:06.832+0000,DEV,Sentiment,undefined,undefined,undefined,undefined,895.8,1149
12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,ALDO,,,,null,MBFEvent.QNAEvent,2018-11-12T11:52:08.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:37:06.832+0000,DEV,QNAEvent,abbandonata,inconsistente,AB,INC,895.75,1152
06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,6LRLPjJDLkQ85QplizAeFS,,,,null,MBFEvent.UserMessage,2018-11-12T11:49:04.561+0000,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T20:37:06.832+0000,DEV,UserMessage,abbandonata,undefined,AB,undefined,895.8,1149


In [ ]:
"""PROCEDIMENTO: 
INPUT: Lettura degli eventi correnti (slot orario corrente)
- giro classico per "event" (quello che già facciamo adesso)
- NUOVO giro per "conversation":
              1.1- lettura della delta table "conversation" (in cui avremo solo type = "previous") e mettiamo tutto in un Dataframe
               1.2- FULL JOIN del Dataframe (quindi dei record della delta table "conversation", solo type = "previous") tra il DF letto e il DF degli eventi correnti, ON conversation.conversationId = event.conversationId (eventi correnti)
               1.3 - determinare il type (F.lit.type) grazie alla FULL JOIN di cui sopra e scrivo in un DF
               1.4 - partendo dal DF del punto 1.3, vado in JOIN con gli eventi correnti per fare le aggregazioni al fine di aggiornare i vari campi della conversazione come ad esempio lo "stato" della conversazione, il "contatore dei messaggi totali", dei "BotMessage", "DataChiusura" etc...
                              N.B. lo "stato" della conversazione sarà ridefinito con la lettura del MAX degli eventi, invece un eventuale contatore (ad esempio TotMessaggi) sarà la Somma del campo letto dalla delta table e la somma del campo determinato dall'aggregazione degli eventi correnti               
N.B. a questo punto avremo conversazioni in Stato "COLD" e "HOT"
               1.5 - inviare al DW quelle che sono diventate COLD ("chiuse" e "abbandonate")
               1.6 - scrittura in Delta Table "Conversation" delle conversazioni rimanenti, ossia le "HOT ("In corso" e "Stand by")
"""

### Taking last event of each Conversation. Rather than ItemId it is considered a Key the couple [ConvId, max(eventime)]

In [64]:
df_1.select('ConversationId').distinct().count()   # 6LRLPjJDLkQ85QplizAeFS, PIPPO e ALDO

In [65]:
new_Events2.select('ConversationId').distinct().count()  #una sola: 6LRLPjJDLkQ85QplizAeFS

In [66]:
display(new_Events2.select('ConversationId').groupBy("ConversationId").count())

In [67]:
new_Events2.createOrReplaceTempView("Events_final")
# è l'ultimo evento di ogni conversazione

last_record = spark.sql(" \
          SELECT ConversationId as convId, max(eventTime) as last_time_conv, min(eventTime) as ConversationStartTime \
          FROM Events_final \
          GROUP BY ConversationId \
          ORDER BY max(eventTime)\
          ")

display(last_record)

convId,last_time_conv,ConversationStartTime
6LRLPjJDLkQ85QplizAeFS,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000
PIPPO,2018-11-12T11:51:04.544+0000,2018-11-12T11:49:04.544+0000
ALDO,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000


####Create Conversations Table: Joining previous cells with Events to take the other fields I need

In [69]:
#cold_conversations= conv_after_stand_by.join(new_Events2, ((new_Events2.conversationId == conv_after_stand_by.convId) & (new_Events2.eventTime == conv_after_stand_by.last_time_conv)
                        #), 'left')

#oppure
conversations = new_Events2.join(last_record, ((new_Events2.conversationId == last_record.convId) & (new_Events2.eventTime == last_record.last_time_conv)
                        ), 'right').select("ConversationId", 'StatusCode', 'Status','last_time_conv', 'RecognitionCode', 'Recognition', 'ConversationStartTime')

conversations= conversations.withColumn('conversationStartDate' , conversations['ConversationStartTime'].cast(StringType()) )

"""def data(x):
  return x[:10]

udf_data= UserDefinedFunction(data, StringType())
conversations= conversations.withColumn('conversationStartDate', udf_data(conversations['conversationStartDate']))"""

#Alternativa senza udf
conversations= conversations.withColumn('conversationStartDate', conversations['conversationStartDate'].cast(StringType()).substr(1,10))

conversations= conversations.select(col("ConversationId"), "ConversationStartDate", 'ConversationStartTime', col('last_time_conv').alias('ConversationEndTime'), 
                                    col('StatusCode').alias('ConversationStatusCode'), col('Status').alias('ConversationStatus'), 
                         col('RecognitionCode').alias('ConversationRecognitionCode'), col('Recognition').alias('ConversationRecognition'))


display(conversations)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition
PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000,AB,abbandonata,undefined,undefined
ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000,AB,abbandonata,INC,inconsistente
6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000,AB,abbandonata,undefined,undefined


In [70]:
conversations.printSchema()

root
-- ConversationId: string (nullable = true)
-- ConversationStartDate: string (nullable = true)
-- ConversationStartTime: timestamp (nullable = true)
-- ConversationEndTime: timestamp (nullable = true)
-- ConversationStatusCode: string (nullable = true)
-- ConversationStatus: string (nullable = true)
-- ConversationRecognitionCode: string (nullable = true)
-- ConversationRecognition: string (nullable = true)

In [71]:
"""for i in range(1, 18):
    if i<=0:
      pass
    if i>=1 and i<=9:
      try:
        print('There are {} files in this day'.format(len(dbutils.fs.ls("/delta/"+'QA'+"/Events/eventDate=2018-12-0" +str(i) ))))
      except:
        print('there are no files for day ', i)
    elif i>=10:
      try:
        print('There are {} files in this day'.format(len(dbutils.fs.ls("/delta/"+'QA'+"/Events/eventDate=2018-12-" +str(i) ))))
      except:
        print('there are no files for day ', i)"""

Out[ 93 ]: 'for i in range(1, 18):\n if i<=0:\n pass\n if i>=1 and i<=9:\n try:\n print(\'There are {} files in this day\'.format(len(dbutils.fs.ls("/delta/"+\'QA\'+"/Events/eventDate=2018-12-0" +str(i) ))))\n except:\n print(\'there are no files for day \', i)\n elif i>=10:\n try:\n print(\'There are {} files in this day\'.format(len(dbutils.fs.ls("/delta/"+\'QA\'+"/Events/eventDate=2018-12-" +str(i) ))))\n except:\n print(\'there are no files for day \', i)'

#### Conversations of last hour (only the ID of the distinct ones)

In [73]:
#display(new_Events2.select('ConversationId').groupBy("ConversationId").count())
lastHourConversations= conversations.select(col('conversationId').alias('NewConvId'), col('ConversationStartDate').alias('NewStartDate'), col('ConversationStartTime').alias('NewStartTime'),
                             col('ConversationEndTime').alias('NewEndTime') ).distinct()
display(lastHourConversations)

NewConvId,NewStartDate,NewStartTime,NewEndTime
6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000
PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000
ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000


### Import previous conversations from delta. Union previous and new

In [75]:
previous_conversations = spark.read.format("delta").load("/delta/"+environment+"/Conversations/")
previous_conversations = previous_conversations.withColumn("recordType", F.lit("previous conv"))
"""# Union of two DataFrames
#unionDataFrame = previous_events.unionAll(df_1)"""
display(previous_conversations)   

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined,previous conv
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined,previous conv
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined,previous conv


### Step 1.2: FULL join between previous conversations and new (la left mantiene i record della prima, inner quelli comuni, la full tutti. Mi prendo conv vecchie, conv nuove, e con full join le ottengo tutte)

In [77]:
#previous_conversations ha 5 record (2 duplicati proprio, errore dei blob). 4 conversazioni: 777Qsjs3i8qJyBr95zABbk (2 duplicati dei record, non solo della conv), 2cpS4QCdLps7ycDeAxodOz, JEBXXIfay3m328mWQLvL0x, 8mkbRBVLXnqB1DyyZ1pVcZ.
# quelle dello slot orario runnato sono 4, e le distinct sono: 6LRLPjJDLkQ85QplizAeFS, ALDO e PIPPO (che ne ha 2). quindi in totale 7 conversazioni distinct.
# last hour conversations è df che ha 3 records, relativi alle 3 unique conversationId. Facendo full join tra previous_conversations e last_hour_conv avrò 8 records:
#le 5 di partenza (previous), più le 3 del nuovo slot orario

In [78]:
#display(previous_conversations.join(lastHourConversations, on= 'ConversationId', how='full'))

allConversations= previous_conversations.join(lastHourConversations, previous_conversations.ConversationId == lastHourConversations.NewConvId, how='full')

display(allConversations) 

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,NewConvId,NewStartDate,NewStartTime,NewEndTime
null,null,null,null,null,null,null,null,null,6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined,previous conv,null,null,null,null
null,null,null,null,null,null,null,null,null,ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000
null,null,null,null,null,null,null,null,null,PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null


In [79]:
allConversations.count()

Out[ 97 ]: 8

#### Adding field recordType to indicate whether the conversation is new or not.

In [81]:
allConversations= allConversations.withColumn('recordType', when(allConversations['ConversationId'].isNull(), F.lit('new conv')).otherwise(allConversations['recordType']))
allConversations= allConversations.withColumn('ConversationId', when(allConversations['ConversationId'].isNull(), allConversations['NewConvId']).otherwise(allConversations['ConversationId']))
allConversations= allConversations.withColumn('ConversationStartDate', when(allConversations['ConversationStartDate'].isNull(), allConversations['NewStartDate']).otherwise(allConversations['ConversationStartDate']))
allConversations= allConversations.withColumn('ConversationStartTime', when(allConversations['ConversationStartTime'].isNull(), allConversations['NewStartTime']).otherwise(allConversations['ConversationStartTime']))
allConversations= allConversations.withColumn('ConversationEndTime', when(allConversations['ConversationEndTime'].isNull(), allConversations['NewEndTime']).otherwise(allConversations['ConversationEndTime']))

allConversations= allConversations.select([col for col in allConversations.columns if col not in ['NewStartDate', 'NewStartTime', 'NewEndTime']])
display(allConversations)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,NewConvId
6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000,null,null,null,null,new conv,6LRLPjJDLkQ85QplizAeFS
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined,previous conv,null
ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000,null,null,null,null,new conv,ALDO
PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000,null,null,null,null,new conv,PIPPO
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined,previous conv,null
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined,previous conv,null


In [82]:
"""new_Events2.createOrReplaceTempView("EventsFinal")
LastItemPerConv = spark.sql(" \
          SELECT ConversationId, max(eventTime) \
          FROM EventsFinal \
          GROUP BY ConversationId\
          ORDER BY max(eventTime)\
          ")
display(LastItemPerConv)"""
# Altro modo, penso più performante perchè non usa vista e spark.sql
LastItemPerConv = new_Events2.select('ConversationId', 'eventTime', 'Status').groupby('ConversationId').agg(F.max(new_Events2.eventTime)).select('*')
#display(new_Events2.select('ConversationId', 'eventTime', 'Status').groupby('ConversationId').agg({"eventTime": "max"}))  #F.max o :"max" così sono uguali
LastItemPerConv= LastItemPerConv.withColumnRenamed('max(eventTime)', 'eventTime')
display(LastItemPerConv)

ConversationId,eventTime
6LRLPjJDLkQ85QplizAeFS,2018-11-12T11:49:04.561+0000
PIPPO,2018-11-12T11:51:04.544+0000
ALDO,2018-11-12T11:52:08.561+0000


In [83]:
LastRecordPerConv= LastItemPerConv.join(new_Events2, on= ['ConversationId', 'eventTime'], how= 'left').withColumnRenamed('ConversationId', 'EventsConvId')
allConversationsUpdated = allConversations.join(LastRecordPerConv, (allConversations.ConversationId == LastRecordPerConv.EventsConvId), how= 'left')

display(LastRecordPerConv)

EventsConvId,eventTime,itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,HourFromLastEvent,EventHourMinute
PIPPO,2018-11-12T11:51:04.544+0000,6789,,,,,,,,,,,0.5085391,,pat,,,,null,MBFEvent.Sentiment,2018-11-12,,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T21:25:19.164+0000,DEV,Sentiment,abbandonata,undefined,AB,undefined,896.57,1151
ALDO,2018-11-12T11:52:08.561+0000,12345,,,,,,,ChitChat,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,,,,null,MBFEvent.QNAEvent,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T21:25:19.164+0000,DEV,QNAEvent,abbandonata,inconsistente,AB,INC,896.55,1152
6LRLPjJDLkQ85QplizAeFS,2018-11-12T11:49:04.561+0000,06470259-e671-11e8-88dc-9fb96e7465b9,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000314,pat,,,,null,MBFEvent.UserMessage,2018-11-12,true,,,,,,,,,,,null,,effettuerò le prove,,,2018-11-12T11:49:00.428+0000,,,77F973BD0EEFAB79927F481B0611EDF8,918449a975df2c63c5a35aa56d415e4f6228035,username,appinsights-wo,wo,new,null,null,null,2018-12-19T21:25:19.164+0000,DEV,UserMessage,abbandonata,undefined,AB,undefined,896.6,1149


In [84]:
allConversationsUpdated.persist()

In [85]:
allConversationsUpdated.columns

Out[ 102 ]: 
['ConversationId',
 'ConversationStartDate',
 'ConversationStartTime',
 'ConversationEndTime',
 'ConversationStatusCode',
 'ConversationStatus',
 'ConversationRecognitionCode',
 'ConversationRecognition',
 'recordType',
 'NewConvId',
 'EventsConvId',
 'eventTime',
 'itemId',
 'ClientAtHome',
 'LUISentities',
 'LUISintent',
 'LUISscore',
 'ProblemSolved',
 'QNAanswer',
 'QNAcategory',
 'QNAscore',
 'QNAsource',
 'QNAuserMessage',
 'TXTANALYTICSscore',
 'activityId',
 'channel',
 'currentScenario',
 'diagnosticResponse',
 'dialogCloseType',
 'endTimeAPICall',
 'eventName',
 'eventdate',
 'isAuthenticated',
 'isLeaf',
 'mainScenario',
 'modemAlignment',
 'modemChanged',
 'modemType',
 'notUnderstoodMessage',
 'reason',
 'scenario',
 'scenarioCalled',
 'scenarioId',
 'startTimeAPICall',
 'telemetryId',
 'text',
 'ticket',
 'ticketForm',
 'timestamp',
 'toolExec',
 'toolResult',
 'user',
 'userId',
 'userName',
 'applicationInsight',
 'businessUnit',
 'type',
 'apiTimeDiff',
 'BotAnswerDuration',
 'EventTimePrevious',
 'DateUTCInsertRow',
 'Environment',
 'eventNameDecode',
 'Status',
 'Recognition',
 'StatusCode',
 'RecognitionCode',
 'HourFromLastEvent',
 'EventHourMinute']

In [86]:
allConversationsUpdated= allConversationsUpdated.select([col for col in allConversations.columns + ['Status', 'Recognition', 'StatusCode', 'RecognitionCode','DateUTCInsertRow'] ]).withColumn('ConversationStatus', when(allConversationsUpdated.Status.isNotNull(), allConversationsUpdated.Status).otherwise(allConversationsUpdated.ConversationStatus))

allConversationsUpdated= allConversationsUpdated.withColumn('ConversationRecognitionCode', when(allConversationsUpdated.RecognitionCode.isNotNull(), allConversationsUpdated.RecognitionCode).otherwise(allConversationsUpdated.ConversationRecognitionCode)).withColumn('ConversationRecognition', when(allConversationsUpdated.Recognition.isNotNull(), allConversationsUpdated.Recognition).otherwise(allConversationsUpdated.ConversationRecognition)).withColumn('ConversationStatusCode', when(allConversationsUpdated.StatusCode.isNotNull(), allConversationsUpdated.StatusCode).otherwise(allConversationsUpdated.ConversationStatusCode))

allConversationsUpdated= allConversationsUpdated.select([col for col in allConversations.columns + ['DateUTCInsertRow'] if col!= 'NewConvId'])
display(allConversationsUpdated)

# DateUTCInsertRow è valorizzato solo per quelle nuove, solamente perchè per quelle vecchie il codice era diverso...

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,DateUTCInsertRow
6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000,AB,abbandonata,undefined,undefined,new conv,2018-12-19T21:29:13.074+0000
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined,previous conv,null
ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000,AB,abbandonata,INC,inconsistente,new conv,2018-12-19T21:29:13.074+0000
PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000,AB,abbandonata,undefined,undefined,new conv,2018-12-19T21:29:13.074+0000
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined,previous conv,null
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined,previous conv,null


In [87]:
##########################################################################################

### Binario: separo Conversations Hot e Cold

In [89]:
hotConversations= allConversationsUpdated.filter((allConversationsUpdated.ConversationStatus!= 'abbandonata') & (allConversationsUpdated.ConversationStatus!='chiusa'))
coldConversations= allConversationsUpdated.filter((allConversationsUpdated.ConversationStatus=='abbandonata') | (allConversationsUpdated.ConversationStatus=='chiusa'))
                                              
display(coldConversations)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,DateUTCInsertRow
6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000,AB,abbandonata,undefined,undefined,new conv,2018-12-19T21:35:28.102+0000
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined,previous conv,null
ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000,AB,abbandonata,INC,inconsistente,new conv,2018-12-19T21:35:28.102+0000
PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000,AB,abbandonata,undefined,undefined,new conv,2018-12-19T21:35:28.102+0000
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,previous conv,null
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined,previous conv,null
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined,previous conv,null


In [90]:
hotConversations_noUTC= hotConversations.select([col for col in hotConversations.columns if col != 'DateUTCInsertRow'])
coldConversations_noUTC= coldConversations.select([col for col in coldConversations.columns if col != 'DateUTCInsertRow'])

In [91]:
coldConversations_noUTC.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Conversations/")  ##Siccome hotConversations è vuota (è Hot se è negli ultimi 20-30 minuti...ma sto runnando orari piu vecchi quindi saranno tutte cold..), sovrascrivo sulla delta Cold, così la riempio. Sennò rimane con zero records...

In [92]:
%sql  -- Ne ha scritte 8 in totale, tutte quelle cold di sopra
SELECT count(*)
FROM DEV.Conversations

In [93]:
#write in delta table in OVERWRITE   DEV.Conversation
#hotConversations.select([col for col in hotConversations.columns if col!= 'DateUTCInsertRow' ]).write.format("delta").mode("overwrite").save("/delta/"+environment+"/Conversations/")
hotConversations_noUTC.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Conversations/")

In [94]:
#spark.sql select per far matchare campi con quelli del DataWharehouse
hotConversations.createOrReplaceTempView("hotConversations")

sqlConversationsHot = spark.sql("""\
SELECT ConversationId
, ConversationStartDate
, ConversationStartTime
, ConversationEndTime
, ConversationStatusCode
, ConversationStatus
, ConversationRecognitionCode
, ConversationRecognition
, 0 as Count_BotMessage
, 0 as Count_UserMessage
, 0 as Count_NotUnderstood
, 0 as Count_Understood
, 0 as Count_LeafEvent
, 0 as Count_Ticket
, 0 as Count_CloseEvent
, 0 as Count_CloseByBot
, '' as DateUTCInsertRow               -- 11/12/2018
, '' as DateUTCInsertRow_Europe_Rome   -- 11/12/2018
FROM hotConversations
""")

#display(sqlConversationsHot)

In [95]:
#spark.sql select per far matchare campi con quelli del DataWharehouse
coldConversations.createOrReplaceTempView("coldConversations")

sqlConversationsCold = spark.sql("""\
SELECT ConversationId
, ConversationStartDate
, ConversationStartTime
, ConversationEndTime
, ConversationStatusCode
, ConversationStatus
, ConversationRecognitionCode
, ConversationRecognition
, 0 as Count_BotMessage
, 0 as Count_UserMessage
, 0 as Count_NotUnderstood
, 0 as Count_Understood
, 0 as Count_LeafEvent
, 0 as Count_Ticket
, 0 as Count_CloseEvent
, 0 as Count_CloseByBot
, DateUTCInsertRow as DateUTCInsertRow              -- 11/12/2018
, '' as DateUTCInsertRow_Europe_Rome  -- 11/12/2018
FROM coldConversations
""")

display(sqlConversationsCold)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,Count_BotMessage,Count_UserMessage,Count_NotUnderstood,Count_Understood,Count_LeafEvent,Count_Ticket,Count_CloseEvent,Count_CloseByBot,DateUTCInsertRow,DateUTCInsertRow_Europe_Rome
6LRLPjJDLkQ85QplizAeFS,2018-11-12,2018-11-12T11:49:04.561+0000,2018-11-12T11:49:04.561+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T21:44:05.027+0000,
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined,0,0,0,0,0,0,0,0,null,
ALDO,2018-11-12,2018-11-12T11:52:08.561+0000,2018-11-12T11:52:08.561+0000,AB,abbandonata,INC,inconsistente,0,0,0,0,0,0,0,0,2018-12-19T21:44:05.027+0000,
PIPPO,2018-11-12,2018-11-12T11:49:04.544+0000,2018-11-12T11:51:04.544+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T21:44:05.027+0000,
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,null,
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,null,
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,null,
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,null,


In [96]:
%sql

SET spark.databricks.queryWatchdog.enabled=false

In [97]:
#WRITE IN DW in table [ConversationCold]  (in APPEND)
sqlConversationsCold.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "ConversationCold").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()

In [98]:
#WRITE IN DW in table [ConversationHot]  (in OVERWRITE)
sqlConversationsHot.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "ConversationHot").option("forward_spark_azure_storage_credentials","True").mode("overwrite").option("tempdir", tempdatabricks_wo).save()

In [99]:
"""%sql
DESCRIBE FORMATTED DEV.Events  --per sapere schema """
# PORTO AVANTI SOLAMENTE CONVERSATIONHOT --> VADO IN JOIN CON TABELLA EVENTS PER PRENDERE SOLO GLI EVENTI DELLE CONVERSAZIONI HOT. POI SOVRASCRIVO QUESTI NELLA DELTA

eventsHot= new_Events2.join(hotConversations_noUTC, on=['ConversationId'], how= 'inner')
eventsHot= eventsHot.select([col for col in new_Events2.columns if col not in hotConversations_noUTC.drop('ConversationId').columns])
display(eventsHot)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,HourFromLastEvent,EventHourMinute


In [100]:
#display(coldConversations_noUTC.distinct())    #vecchio

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,new conv
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,new conv


In [101]:
#coldConversations_noUTC.select('ConversationId').distinct().count()
#display(coldConversations_noUTC.select('ConversationId').distinct())   #vecchio

ConversationId
6LRLPjJDLkQ85QplizAeFS


In [102]:
#analogo a eventsHot
#eventsCold= new_Events2.join(coldConversations_noUTC, on= ['ConversationId'], how= 'inner')
eventsCold= new_Events2.join(coldConversations_noUTC, on=['ConversationId'], how= 'inner')
eventsCold= eventsCold.select([col for col in new_Events2.columns if col not in coldConversations_noUTC.drop('ConversationId').columns])
#display(eventsCold)

In [103]:
eventsCold.count()  # --> mi sovrascrive i 4 records nuovi

Out[ 108 ]: 4

In [104]:
eventsCold.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Events_Federico/")  #Siccome eventsHot è vuota (è Hot se è negli ultimi 20-30 minuti...ma sto runnando orari piu vecchi quindi saranno tutte cold..), sovrascrivo sulla delta eventsCold, così la riempio. Sennò rimane con zero records...

In [105]:
#write in delta table (in OVERWRITE)
eventsHot.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Events/") #29/10/2018: output changed from "events" to "Events"

In [106]:
#spark.sql select per far matchare campi con quelli del DataWharehouse
eventsCold.createOrReplaceTempView("eventsCold")

sqlEventsCold = spark.sql("""
SELECT
  '' as ID
, CAST(eventTime as timestamp) as EventTime
, year(eventTime) * 10000 + month(eventTime) * 100 + day(eventTime) as EventDateNum
, hour(eventTime) as EventHourNum
, minute(eventTime) as EventMinuteNum
, second(eventTime) as EventSecondNum
, EventName as EventName
, '' as internalID
, '' as ClientIP
, '' as Continent
, '' as Country
, '' as Province
, '' as City
, LEFT(Text,4000) as Text
, ConversationId as ConversationId
, Channel
, Timestamp
, UserName
, UserId
, LUISintent
, LUISscore
, MainScenario
, Reason
, Scenario
, CurrentScenario
, QNAscore as QNAscore
, QNAanswer as QNAanswer
, QNAuserMessage as QNAuserMessage
, QNAsource as QNAsource
, QNAcategory as QNAcategory
, ItemId as ItemId
, ActivityId as ActivityId
, BotAnswerDuration as DurationAnswerBot
, CAST(startTimeAPICall as timestamp) as startTimeAPICall
, CAST(endTimeAPICall as timestamp) as endTimeAPICall
, apiTimeDiff as DurationAnswerAPI
, '' as modem
, '' as linea
, ticket as ticket
, ticketForm as ticketForm
, ClientAtHome as ClientAtHome
, modemType as modemType
, diagnosticResponse as diagnosticResponse
, modemAlignment as modemAlignment
, toolExec as toolExec
, toolResult as toolResult
, TXTANALYTICSscore as TXTANALYTICSscore    -- 03/12/2018 converted from string to float
, scenarioid as scenarioid
, scenariocalled as scenariocalled
, isLeaf as isLeafText
, DateUTCInsertRow as DateUTCInsertRow
, modemChanged as modemChanged
, dialogCloseType as dialogCloseType
, EventTimePrevious as EventTimePrevious
, notUnderstoodMessage as notUnderstoodMessage
, applicationInsight as applicationInsight
, businessUnit as businessUnit
, problemSolved as ProblemSolved                      -- before "ProblemSolved", after "problemSolved"
, user as user
, isAuthenticated as isAuthenticated
, '' as Status -- added on 2018-11-23
, '' as Recognition -- added on 2018-11-23
, '' as StatusCode -- added on 2018-11-23
, '' as RecognitionCode -- added on 2018-11-23
, 0 as HourFromLastEvent -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, 0 as EventHourMinute -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, Environment as Environment
, eventNameDecode as eventNameDecode
FROM eventsCold
""")

In [107]:
#WRITE IN DW in table [eventsCold]  (in APPEND)
sqlEventsCold.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "EventCold").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()

In [108]:
#PREPARING RESULTSET TO WRITE INTO DW
# ATTENTION: NUMBER OF COLUMNS MUST MATCH WITH OUTPUT AND IN THE SAME ORDER OF DB TABLE STRUCTURE


# Create or Replace TempView, uncomment if sql manipulation must be used
eventsHot.createOrReplaceTempView("eventsHot")

sqlEventsHot = spark.sql("""
SELECT
  '' as ID
, CAST(eventTime as timestamp) as EventTime
, year(eventTime) * 10000 + month(eventTime) * 100 + day(eventTime) as EventDateNum
, hour(eventTime) as EventHourNum
, minute(eventTime) as EventMinuteNum
, second(eventTime) as EventSecondNum
, EventName as EventName
, '' as internalID
, '' as ClientIP
, '' as Continent
, '' as Country
, '' as Province
, '' as City
, LEFT(Text,4000) as Text
, ConversationId as ConversationId
, Channel
, Timestamp
, UserName
, UserId
, LUISintent
, LUISscore
, MainScenario
, Reason
, Scenario
, CurrentScenario
, QNAscore as QNAscore
, QNAanswer as QNAanswer
, QNAuserMessage as QNAuserMessage
, QNAsource as QNAsource
, QNAcategory as QNAcategory
, ItemId as ItemId
, ActivityId as ActivityId
, BotAnswerDuration as DurationAnswerBot
, CAST(startTimeAPICall as timestamp) as startTimeAPICall
, CAST(endTimeAPICall as timestamp) as endTimeAPICall
, apiTimeDiff as DurationAnswerAPI
, '' as modem
, '' as linea
, ticket as ticket
, ticketForm as ticketForm
, ClientAtHome as ClientAtHome
, modemType as modemType
, diagnosticResponse as diagnosticResponse
, modemAlignment as modemAlignment
, toolExec as toolExec
, toolResult as toolResult
, TXTANALYTICSscore as TXTANALYTICSscore    -- 03/12/2018 converted from string to float
, scenarioid as scenarioid
, scenariocalled as scenariocalled
, isLeaf as isLeafText
, DateUTCInsertRow as DateUTCInsertRow
, modemChanged as modemChanged
, dialogCloseType as dialogCloseType
, EventTimePrevious as EventTimePrevious
, notUnderstoodMessage as notUnderstoodMessage
, applicationInsight as applicationInsight
, businessUnit as businessUnit
, problemSolved as ProblemSolved                      -- before "ProblemSolved", after "problemSolved"
, user as user
, isAuthenticated as isAuthenticated
, '' as Status -- added on 2018-11-23
, '' as Recognition -- added on 2018-11-23
, '' as StatusCode -- added on 2018-11-23
, '' as RecognitionCode -- added on 2018-11-23
, 0 as HourFromLastEvent -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, 0 as EventHourMinute -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, Environment as Environment
, eventNameDecode as eventNameDecode
FROM eventsHot
""")

In [109]:
###WRITE IN DW in table [EventHot] (NB: now write option is "Append" but when will have Hot and Cold logic it will be "Overwrite" in Hot Table and "Append" in Cold Table)
#WRITE IN DW in table [EventHot]  (in OVERWRITE)
sqlEventsHot.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "EventHot").option("forward_spark_azure_storage_credentials","True").mode("overwrite").option("tempdir", tempdatabricks_wo).save()

In [110]:
allConversationsUpdated.unpersist() #dbutils.fs.rm("/delta/DEV/Events/", True)

In [111]:
#command to manage "user authenticated"
new_Events2.filter(new_Events2.type == 'new').createOrReplaceTempView("finalTempTable")

#select for "user authenticated"
sqlQueryResult_User = spark.sql("""
SELECT
ConversationId as ConversationId
, user as user
-- DateUTCInsertRow as DateUTCInsertRow,    --to be added
FROM finalTempTable
WHERE
isAuthenticated = 'true' and (user is not null or user <> '')
GROUP BY
ConversationId, user
""")

#WRITE IN DW in table [DIM_User]
sqlQueryResult_User.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "DIM_User").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()

In [112]:
"""%sql
OPTIMIZE QA.Events --Dalta Table "Events" is partitioned by "eventDate"
WHERE EventDate >= current_timestamp() - INTERVAL 1 day
ZORDER BY (EventName, ConversationId)"""

Out[ 73 ]: '%sql\nOPTIMIZE QA.Events --Dalta Table "Events" is partitioned by "eventDate"\nWHERE EventDate >= current_timestamp() - INTERVAL 1 day\nZORDER BY (EventName, ConversationId)'

In [113]:
##Andrea 26/11/2018: Optimize disabled because now is scheduled in a specific notebook wich runs daily during the night
#optimize = spark.sql("OPTIMIZE {}.Events WHERE EventDate >= current_timestamp() - INTERVAL 1 day ZORDER BY (EventName, ConversationId)".format(environment))